In [1]:
import numpy as np
import pandas as pd
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda, Resize, Compose 
import matplotlib.pyplot as plt
from PIL import Image
import os

In [22]:
# index = pd.read_csv('LEGO/index.csv')
# mat = pd.read_csv('LEGO/metadata.csv')
# test = pd.read_csv('LEGO/test.csv')

In [59]:
test

,path,class_id
0,test/001.jpg,32
1,test/002.jpg,32
2,test/003.jpg,32
3,test/004.jpg,32
4,test/005.jpg,1
...,...,...
65,test/066.jpg,34
66,test/067.jpg,34
67,test/068.jpg,35
68,test/069.jpg,35


In [60]:
mat

,class_id,lego_ids,lego_names,minifigure_name
0,1,[76115],['Spider Mech vs. Venom'],SPIDER-MAN
1,2,[76115],['Spider Mech vs. Venom'],VENOM
2,3,[76115],['Spider Mech vs. Venom'],AUNT MAY
3,4,[76115],['Spider Mech vs. Venom'],GHOST SPIDER
4,5,[75208],"[""Yoda's Hut""]",YODA
5,6,[75208],"[""Yoda's Hut""]",LUKE SKYWALKER
6,7,[75208],"[""Yoda's Hut""]",R2-D2
7,8,[75199],"[""General Grievous' Combat Speeder""]",MACE WINDU
8,9,[75199],"[""General Grievous' Combat Speeder""]",GENERAL GRIEVOUS
9,10,[75264],"[""Kylo Ren's Shuttle Microfighter""]",KYLO REN


In [2]:
class LegoImageDataset(Dataset):
    def __init__(self,path='LEGO/index.csv'):
        self.index = pd.read_csv(path)
        self.labels = self.index['class_id']
        self.img = self.index['path']
        self.imgsize = image_size=(244,244)
        self.transform = Compose([Resize(self.imgsize),ToTensor()])
        self.label_transform = Compose([ToTensor()])
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join('LEGO/',self.img[idx])
        image = Image.open(img_path).convert('RGB')
        label = torch.as_tensor(self.labels[idx])
        label = torch.nn.functional.one_hot(label,36)
        image = self.transform(image)
        
        return image,label

In [3]:
# load data
from torch.utils.data import DataLoader

train_data = LegoImageDataset()
test_data = LegoImageDataset(path='LEGO/test.csv')

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=3, shuffle=True)

In [4]:
# Build resnet18
# defining the optimizer

from torchvision import models
import torch.nn as nn
from torch.optim import Adam

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 256)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 36)

optimizer = Adam(model.parameters(), lr=0.0003)

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [42]:
# training model
import torch.nn.functional as F

train_losses = []
# empty list to store validation losses
val_losses = []

for epoch in range(10):
    model.train()
    tr_loss = 0
    total =0
    total_train = 0
    # getting the training set
    x_train, y_train = next(iter(train_dataloader))
    # getting the validation set
    x_val, y_val = next(iter(test_dataloader))
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)
    #print(output_val,y_val)
    
    
    # computing the training and validation loss
    loss_train = F.cross_entropy(output_train, torch.max(y_train, 1)[1])
    loss_val = F.cross_entropy(output_val,  torch.max(y_val, 1)[1])
    
    train_losses.append(loss_train)
    val_losses.append(loss_val)
    # acc of validation
    total += sum(torch.max(output_val, 1)[1]==torch.max(y_val, 1)[1])
    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch)
        print('train_loss',loss_train,'val_loss :', loss_val,'Acc val:',total/y_val.size()[0])
        

Epoch :  1
train_loss tensor(0.0218, grad_fn=<NllLossBackward>) val_loss : tensor(1.0490, grad_fn=<NllLossBackward>) Acc val: tensor(1.)
Epoch :  3
train_loss tensor(0.0172, grad_fn=<NllLossBackward>) val_loss : tensor(0.9589, grad_fn=<NllLossBackward>) Acc val: tensor(1.)
Epoch :  5
train_loss tensor(0.0171, grad_fn=<NllLossBackward>) val_loss : tensor(1.3752, grad_fn=<NllLossBackward>) Acc val: tensor(0.6667)
Epoch :  7
train_loss tensor(0.0212, grad_fn=<NllLossBackward>) val_loss : tensor(0.8757, grad_fn=<NllLossBackward>) Acc val: tensor(1.)
Epoch :  9
train_loss tensor(0.1093, grad_fn=<NllLossBackward>) val_loss : tensor(0.8935, grad_fn=<NllLossBackward>) Acc val: tensor(1.)


In [47]:
# save model
torch.save(model.state_dict(), 'LEGO/weight')

In [10]:
# load model
model.load_state_dict(torch.load('LEGO/weight'))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
# show the test ACC
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)
total = 0
count = 0

lne = test_dataloader
for batch_idx, (data, target) in enumerate(test_dataloader):
    predict = model(data)
    total += sum(torch.max(predict, 1)[1]==torch.max(target, 1)[1])
    count += target.size()[0]

print('ACC=',total/count)

ACC= tensor(0.9286)
